In [1]:
import pymc as pm
import numpy as np
import pandas as pd
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import train_test_split
import extract_correct_csv

print(f"PyMC version: {pm.__version__}")


PyMC version: 4.2.2


## Ridge classifier with sklearn

In [4]:
valid_subject = extract_correct_csv.extract_only_valid_subject()
valid_subject.remove(49)
valid_subject

[2, 4, 10, 41, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55]

In [58]:
columns = ['subject', 'type', 'train accuracy', 'test accuracy']
results = pd.DataFrame(columns = columns)
index = 1
for x in valid_subject:
    string_sub = extract_correct_csv.read_correct_subject_csv(x)
    df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
    y = np.array(list([int (d > 2) for d in df_['rating']]))
    types_ = ['hr', 'eda', 'pupil']
    for type_ in types_:
        X = pd.read_csv('data/features_4_2/'+type_+'/45.csv')
        # creating train and test
        X_train, X_test,\
        y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)
        clf = RidgeClassifier()
        clf.fit(X_train, y_train)
        train_ = clf.score(X_train, y_train)
        test_ = clf.score(X_test, y_test)
        dict_ = {'subject': x, 'type': type_, 'train accuracy': train_, 'test accuracy': test_}
        results = pd.concat([results, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)
        index = index + 1

In [59]:
results.loc[results.groupby(by=['subject'])['train accuracy'].idxmax()]

,subject,type,train accuracy,test accuracy
1,2,eda,0.742188,0.62500
4,4,eda,0.710938,0.43750
7,10,eda,0.765625,0.78125
10,41,eda,0.726562,0.53125
13,43,eda,0.859375,0.71875
17,44,pupil,0.656250,0.53125
19,45,eda,0.804688,0.65625
23,46,pupil,0.679688,0.68750
25,47,eda,0.742188,0.65625
28,48,eda,0.710938,0.56250


In [68]:
results.groupby(by='type')['test accuracy'].mean()

type
eda      0.646484
hr       0.546875
pupil    0.535156
Name: test accuracy, dtype: float64